### Set the corresponding values in the cell below. Afterwards, just run the following cells

In [9]:
# Class label csv path
labels_csv_path = 'datasets/home_labels.csv'
# Video example
video_path = 'datasets/Crying/_ceBK5pQTrs_000033_000043.mp4'

### Import libraries 

In [10]:
from msclap import CLAP
import torch.nn.functional as F
import numpy as np
import torch
import os
from moviepy.editor import VideoFileClip

### Get Audio from video

In [11]:
video = VideoFileClip(video_path)
if video.audio is None:
    print(f"This video {video_path} has no audio!")
else:
    audio_path = 'datasets/audios' + video_path[8:-3] + 'wav'
    audio_dir = "/".join(audio_path.split("/")[:-1])
    print(audio_dir)
    if not os.path.exists(audio_dir):
            os.makedirs(audio_dir)
    audio_arr = video.audio.write_audiofile(audio_path, verbose=False)
    print(f"Audio saved to {audio_path}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

datasets/audios/Crying
MoviePy - Writing audio in datasets/audios/Crying/_ceBK5pQTrs_000033_000043.wav


MoviePy - Done.
Audio saved to datasets/audios/Crying/_ceBK5pQTrs_000033_000043.wav


### Read class Labels

In [12]:
import csv
import numpy as np

label2id = {}
id2label = {}
with open(labels_csv_path, mode='r') as file:
    csv_reader = csv.reader(file)

    for i, row in enumerate(csv_reader):
        class_name = row[0]
        label2id[class_name] = i
        id2label[i] = class_name

class_labels = list(label2id.keys())

print(f"{len(class_labels)} Unique classes: {class_labels}.")

70 Unique classes: ['No people is in the room', 'Studying', 'Typing', 'Using computer', 'Making phone calls', 'Play with phone/tablet', 'Playing with pets', 'Feeding pets', 'Reading', 'Writing', 'Setting table', 'Eating food', 'Dining', 'Cooking', 'Sleeping', 'Brushing teeth', 'Showering', 'Playing music', 'TV', 'Cleaning', 'Doing laundry', 'Mopping floor', 'Vacumning', 'Ironing', 'Organizing space', 'Sewing', 'Knitting', 'Decorating', 'Party', 'Chatting', 'Talking', 'Singing', 'Laughing', 'Speaking', 'Dancing', 'Drinking', 'Stretching', 'Meditating', 'Drawing', 'Painting', 'Playing board games', 'Playing video games', 'Taking photos', 'Potluck', 'Working', 'Exercising', 'Walking', 'Running', 'Celebratin', 'Physical altercations', 'Verbal confrontations', 'Using drug', 'Theft or vandalism', 'Fighting', 'Domestic violence', 'Break in', 'Glass breaking', 'Fire accident', 'Fire alarm', 'Unattended cooking', 'Open flame', 'Smoking', 'Gunshot', 'Making noise', 'Falling down', 'Tripping', 'C

### CLAP inference with given audio

In [13]:
with torch.no_grad():
    # Load model (Choose between versions '2022' or '2023')
    # The model weight will be downloaded automatically if `model_fp` is not specified
    clap_model = CLAP(version = '2023', use_cuda=False)

    # Extract text embeddings
    text_embeddings = clap_model.get_text_embeddings([f"This is a sound of {c}"for c in class_labels])

    # Extract audio embeddings
    audio_embeddings = clap_model.get_audio_embeddings([audio_path])

    # Compute similarity between audio and text embeddings 
    similarities = clap_model.compute_similarity(audio_embeddings, text_embeddings)

similarity = F.softmax(similarities, dim=1)
values, indices = similarity[0].topk(5)
pred_index = indices[0]

# Print the results
print("Ground Truth: {}".format(video_path.split('/')[-2]))
print("Top predictions:\n")
for value, index in zip(values, indices):
    print(f"{class_labels[index]:>16s}: {100 * value.item():.2f}%")

Ground Truth: Crying
Top predictions:

          Crying: 73.72%
     Suffocating: 13.27%
No people is in the room: 2.11%
   Doing laundry: 1.95%
Playing with pets: 1.87%
